# Generator of C lang samples (all of them I guess)

In [ ]:
int main() {
    char napis[6];
    printf("Podaj napis: ");
    if(scanf("%5s", napis)!=1)
    {
        printf("Niepoprawny format napisu!");
        return 1;
    }
    napis[0] = toupper(napis[0]);
    napis[1] = toupper(napis[1]);
    napis[2] = toupper(napis[2]);
    napis[3] = toupper(napis[3]);
    napis[4] = toupper(napis[4]);
    printf("%s", napis);
    return 0;
}

In [3]:
import numpy as np
import random

## version 1.0

There is always type, an ID and openning curly brace at the beginning that simulates a function definition. Function can have parameters.


While generating I assume that if some ID is defined than it is defined "in order", it means that there should not be a situation when ID2 is defined before ID1 has been defined


In [4]:
beg_call = "("
end_call = ")"
beg_block = "{"
end_block = "}"

built_in_types = ["int", "char"]
types_to_return_from_function = built_in_types + ["void"]

math_ops = ["-", "+", "/", "*"]
EMPTY = "EMPTY"
static_values_tokens = ["STRING", "NUM"]

vocabulary = [
  "void", "int", "char", "STRING", "NUM", "return", "printf", "scanf", 
  "-", "+", "/", "*", "&", "=", "(", ")", ";", "{", "}", "EMPTY"
]

IDs = []
funs = []

for index in range(8):
  current_id = "ID" + str(index)
  vocabulary.append(current_id)
  IDs.append(current_id)

# will add functions in some further generation
# for index in range(8):
#   current_fun = "FUN" + str(index)
#   vocabulary.append(current_fun)
#   funs.append(current_fun)

print(len(vocabulary))
print(vocabulary)
print(IDs)
print(funs)

28
['void', 'int', 'char', 'STRING', 'NUM', 'return', 'printf', 'scanf', '-', '+', '/', '*', '&', '=', '(', ')', ';', '{', '}', 'EMPTY', 'ID0', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'ID7']
['ID0', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'ID7']
[]


In [46]:
#
# defined_ids is a tuple -> (type, ID, definition_place)
# definition_place function-:
#   0: -body
#   1: -argument
#   2: -name


def val_token_dep_on_type(type):
  if type == "char":
    return "STRING"
  if type == "int":
    return "NUM"
  raise Exception(f"No known type '{type}'")

def get_first_free_ID(defined_ids):
  if len(defined_ids) >= len(IDs):
    raise Exception(f"hueheu, cannot give you another ID cause you already have {len(defined_ids)}")
  return IDs[len(defined_ids)]

def arg_pair(defined_ids):
  type = random.choice(built_in_types)
  return type, (type, get_first_free_ID(defined_ids), 1)

def gen_args(defined_ids, no_of_args):
  args = []
  ids = []
  for _ in range(no_of_args):
    type, id = arg_pair(defined_ids + ids)
    ids.append(id)
    args += [type, id[1]] + [","]

  # removing comma (',')
  if len(args):
    del args[-1]

  return args, ids

def function_start():
  function_type = random.choice(types_to_return_from_function)
  to_return = [function_type, IDs[0], "("]
  defined_ids = [(function_type, IDs[0], 2)]
  no_of_args = random.randint(0, len(IDs) // (len(IDs) ** 0.5))
  args, ids = gen_args(defined_ids, no_of_args)
  
  defined_ids += ids
  to_return += args + [")", "{"]
  return to_return, defined_ids

# will add functions in some further generation
# returns list of ID that are passed as arguments
def call(defined_ids):
  to_return = []
  calls_choices = ["printf", "scanf"] # for now and this whole function will change when I add function calls (or just expression function to use here)
  chosen_call = random.choice(calls_choices)
  chosen_value = random.choice(static_values_tokens + [id[1] for id in defined_ids if id[2] != 2])
  to_return += [chosen_call, "(", "STRING", ",", chosen_value, ")", ";"]
  return to_return, []

# can be called only if there are any IDs defined already (not as arguments)
# this function is probably going to be the most frequently changed part of this generation
def assignment(defined_ids):
  chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  if definition_place == 0:
    return [], [] # cycling
  while(chosen_type == "void"):
    if len(defined_ids) == 1:
      return definition(defined_ids)
    chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  return [chosen_id, "=", val_token_dep_on_type(chosen_type), ";"], []

# returns ID that has been defined
def definition(defined_ids):
  if len(defined_ids) >= len(IDs):
    return [], [] # cycling
  id = get_first_free_ID(defined_ids)
  type = random.choice(built_in_types)
  if random.random() > 0.5: # no assignment
    tokens = [type, id, ";"]
  else:
    tokens = [type, id, "=", val_token_dep_on_type(type), ";"]
  return tokens, [(type, id, 1)]

# TODO: here I should consider to add operations, mathematical with ids and so on
expression_depth = 0
def expression(defined_ids):
  to_return = []
  
  return to_return, []

def gen_actions(defined_ids, no_of_actions):
  to_return = []
  actions = [call, definition, assignment, expression]
  for _ in range(no_of_actions):
    action = random.choice(actions)
    tokens, new_ids = action(defined_ids)
    to_return += tokens
    defined_ids += new_ids
    
  return to_return, defined_ids

def c_v1_simple_generate_samples(no_of_samples):
  to_return = []
  # for _ in range(no_of_samples):
  no_of_actions = random.randint(1, 2)
  tokens, defined_ids = function_start()
  to_return += tokens
  to_return += gen_actions(defined_ids, no_of_actions)[0]
  to_return += ["return", "NUM", ";", "}"] # end of function
  return to_return


def fill_vector_with_empty_tokens():
  pass

In [57]:
generated = c_v1_simple_generate_samples(1)
print(generated)
" ".join(generated)

['int', 'ID0', '(', 'char', 'ID1', ',', 'int', 'ID2', ')', '{', 'scanf', '(', 'STRING', ',', 'STRING', ')', ';', 'return', 'NUM', ';', '}']


'int ID0 ( char ID1 , int ID2 ) { scanf ( STRING , STRING ) ; return NUM ; }'

In [48]:
for _ in range(10000):
  c_v1_simple_generate_samples(1)